In [25]:
import os
import json
import pandas as pd
import numpy as np
import torch
from torch import nn
import plotly.graph_objects as go

def plot_3d_response_pf_ratio_surface(
    run_folder: str,
    fold: int,
    test_csv: str,
    sample_index: int,
    fluid_range: tuple[float,float],
    pf_ratio_range: tuple[float,float],
    n_fluid: int = 50,
    n_ratio: int = 50,
    show_GEL: bool = True,
    show_SLKW: bool = True,
    show_XLINK: bool = True
):
    """
    Interactive 3D surfaces per fluid type:
      X = Total.Fluid
      Y = Propellant/Fluid ratio
      Z = predicted Output1
    Marks the actual sample point.
    Reads `include_ratio_features` from hyperparams.json.
    `show_GEL`, `show_SLKW`, `show_XLINK` toggle which fluids to include.
    """
    # Column names
    gpi_col           = "GPI (gross perforated interval ft)"
    prop_per_gpi_col  = "Proppant.per.GPI..lb.ft."
    fluid_per_gpi_col = "Fluid.per.GPI..gal.ft."
    total_prop_col    = "Total.Proppant.Volume"
    total_fluid_col   = "Total.Fluid"
    fluid_type_col    = "Fluid.Type"
    output1_col       = "BOE_Prodoction_2 year cum"

    # Load hyperparams & norms
    run_id = os.path.basename(os.path.normpath(run_folder))
    hp = json.load(open(os.path.join(run_folder, f"{run_id}_hyperparams.json")))
    norms = json.load(open(os.path.join(run_folder, f"{run_id}_norms.json")))

    layer_dims = hp["layer_dims"]
    activations = hp["activations"]
    include_ratio_features = hp.get("include_ratio_features", True)

    # Handle norms y_mean/y_std possibly list
    raw_mean = norms["y_mean"]
    raw_std  = norms["y_std"]
    y1_mean = raw_mean[0] if isinstance(raw_mean, (list,tuple)) else raw_mean
    y1_std  = raw_std[0]  if isinstance(raw_std,  (list,tuple)) else raw_std

    x_mean = norms["x_mean"]
    x_std  = norms["x_std"]

    # Load test data & pick sample
    df = pd.read_csv(test_csv)
    sample = df.iloc[sample_index]
    gpi = float(sample[gpi_col])
    sf  = float(sample[total_fluid_col])
    sp  = float(sample[total_prop_col])
    st  = sample[fluid_type_col]
    y1_true = float(sample[output1_col])
    sample_ratio = sp / sf

    # Build feature lists
    numeric_feats = list(x_mean.keys())
    if not include_ratio_features:
        numeric_feats = [
            c for c in numeric_feats
            if c not in (prop_per_gpi_col, fluid_per_gpi_col)
        ]
    all_types = sorted(df[fluid_type_col].unique())
    # determine which types to plot
    plot_types = []
    if show_GEL:
        plot_types += [t for t in all_types if t.upper()=="GEL"]
    if show_SLKW:
        plot_types += [t for t in all_types if t.upper()=="SLKW"]
    if show_XLINK:
        plot_types += [t for t in all_types if t.upper()=="XLINK"]
    plot_types = list(dict.fromkeys(plot_types))

    # dummy features always full set
    dummy_feats_all = [f"{fluid_type_col}_{t}" for t in all_types]

    # Define & load model
    class MLPNet(nn.Module):
        def __init__(self, in_dim, hidden_dims, activations, out_dim):
            super().__init__()
            layers, dims = [], [in_dim] + hidden_dims
            for i,h in enumerate(hidden_dims):
                layers.append(nn.Linear(dims[i], dims[i+1]))
                act = activations[i].lower()
                if   act=='relu':     layers.append(nn.ReLU())
                elif act=='tanh':     layers.append(nn.Tanh())
                elif act=='sigmoid':  layers.append(nn.Sigmoid())
                elif act=='softplus': layers.append(nn.Softplus())
                else: raise ValueError(f"Unknown activation '{activations[i]}'")
            layers.append(nn.Linear(dims[-1], out_dim))
            self.net = nn.Sequential(*layers)
        def forward(self, x):
            return self.net(x)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = MLPNet(
        in_dim      = len(numeric_feats) + len(dummy_feats_all),
        hidden_dims = layer_dims,
        activations = activations,
        out_dim     = 2
    ).to(device)
    model.load_state_dict(torch.load(
        os.path.join(run_folder, f"{run_id}_fold{fold}.pth"),
        map_location=device
    ))
    model.eval()

    # Build grid in (fluid, ratio)
    fluid_vals = np.linspace(fluid_range[0], fluid_range[1], n_fluid)
    ratio_vals = np.linspace(pf_ratio_range[0], pf_ratio_range[1], n_ratio)
    Fg, Rg = np.meshgrid(fluid_vals, ratio_vals)

    # Baseline features dict
    base = {f: float(sample[f]) for f in numeric_feats}
    for d in dummy_feats_all:
        base[d] = 0.0

    # Create Plotly figure
    fig = go.Figure()

    # Loop per selected fluid type
    for ft in plot_types:
        # set one-hot among full dummy set
        for d in dummy_feats_all:
            base[d] = 1.0 if d == f"{fluid_type_col}_{ft}" else 0.0

        Z = np.zeros_like(Fg)
        for i in range(len(ratio_vals)):
            for j in range(len(fluid_vals)):
                f = fluid_vals[j]
                r = ratio_vals[i]
                p = f * r
                base[total_fluid_col] = f
                base[total_prop_col]  = p
                if include_ratio_features:
                    base[prop_per_gpi_col]  = p / gpi
                    base[fluid_per_gpi_col] = f / gpi
                x_vec = [(base[c]-x_mean[c])/x_std[c] for c in numeric_feats]
                x_vec += [base[d] for d in dummy_feats_all]
                Xi = torch.tensor([x_vec], dtype=torch.float32).to(device)
                with torch.no_grad():
                    out = model(Xi).cpu().numpy().flatten()
                Z[i,j] = float(out[0] * y1_std + y1_mean)

        fig.add_trace(go.Surface(
            x=fluid_vals,
            y=ratio_vals,
            z=Z,
            name=ft,
            showscale=False,
            opacity=0.75
        ))

    # Mark the actual sample point
    fig.add_trace(go.Scatter3d(
        x=[sf], y=[sample_ratio], z=[y1_true],
        mode='markers',
        marker=dict(size=6, color='red'),
        name='actual'
    ))

    fig.update_layout(
        title=f"{output1_col} vs Total Fluid & Propellant/Fluid Ratio",
        scene=dict(
            xaxis_title="Total Fluid",
            yaxis_title="Propellant/Fluid Ratio",
            zaxis_title=output1_col
        ),
        legend_title="Fluid Type"
    )

    # Open in browser
    fig.show(renderer="browser")



In [26]:
fluid_range = (1e5, 2e7)
sample_index=1
pf_ratio_range=(1, 1.4)
test="/home/kamiar/chevron/Eagle-Ford/First/data/Eagle Ford Data(Eagle Ford)_test.csv"

# relu

In [27]:
plot_3d_response_pf_ratio_surface(
    run_folder="/home/kamiar/chevron/Eagle-Ford/Second/d3feed88",
    fold=1,
    test_csv=test,
    sample_index=sample_index,
    fluid_range=fluid_range,
    pf_ratio_range=pf_ratio_range,
    show_GEL=True,
    show_SLKW=False,
    show_XLINK=False
)

gio: http://127.0.0.1:46175: Operation not supported


# softplus

In [28]:
plot_3d_response_pf_ratio_surface(
    run_folder="/home/kamiar/chevron/Eagle-Ford/Second/4eb5e823",
    fold=1,
    test_csv=test,
    sample_index=sample_index,
    fluid_range=fluid_range,
    pf_ratio_range=pf_ratio_range,
    show_GEL=True,
    show_SLKW=False,
    show_XLINK=False
)

gio: http://127.0.0.1:41355: Operation not supported


# relu rdundatn removed

In [29]:
plot_3d_response_pf_ratio_surface(
    run_folder="/home/kamiar/chevron/Eagle-Ford/Second/5044f3a3",
    fold=6,
    test_csv=test,
    sample_index=sample_index,
    fluid_range=fluid_range,
    pf_ratio_range=pf_ratio_range,
    show_GEL=True,
    show_SLKW=False,
    show_XLINK=False
)

gio: http://127.0.0.1:37753: Operation not supported


# redundatn removed softplus

In [32]:
plot_3d_response_pf_ratio_surface(
    run_folder="/home/kamiar/chevron/Eagle-Ford/Second/98a1fcb5",
    fold=6,
    test_csv=test,
    sample_index=sample_index,
    fluid_range=fluid_range,
    pf_ratio_range=pf_ratio_range,
    show_GEL=True,
    show_SLKW=False,
    show_XLINK=False
)

gio: http://127.0.0.1:37405: Operation not supported
